In [6]:
import random
import numpy as np

# Main Constants
NUM_CLIENTS = random.randint(100, 200)
NUM_DATACENTERS = 1
NUM_HOSTS = 2

# Cloud node parameters
CLOUD_NODES = 10
CLOUD_CPU_POWER = (3000, 5000)  # MIPS
CLOUD_RAM = 8  # GB
CLOUD_STORAGE = 1  # TB
CLOUD_BANDWIDTH = 0.5  # Gb/s

# Fog node parameters
FOG_NODES = 10
FOG_CPU_POWER = (1000, 2800)  # MIPS
FOG_RAM = 2  # GB
FOG_STORAGE = 0.5  # TB
FOG_BANDWIDTH = 1  # Gb/s

# Task parameters
TASK_LENGTH = (2000, 56000)  # MI
DATA_SIZE = (400, 600)  # MB
TASK_SETS = [300, 600, 900, 1200, 1500]  # Different sets of task numbers

# Optinizer Constants
C1 = 0.18
C2 = 0.82
MAX_OA = 0.5
MIN_OA = 0.2

class Node:
    def __init__(self, cpu_power, ram, storage, bandwidth):
        self.cpu_power = cpu_power
        self.ram = ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.total_execution_time = 0

    def process_task(self, task_length):
        processing_time = task_length / self.cpu_power
        self.total_execution_time += processing_time
        return processing_time

    def calculate_energy_consumption(self, makespan):
        beta_j = 10**-8 * self.cpu_power**2
        alpha_j = 0.6 * beta_j
        active_energy = self.total_execution_time * beta_j
        idle_energy = (makespan - self.total_execution_time) * alpha_j
        total_energy = active_energy + idle_energy
        return total_energy

def initialize_population(nodes, num_tasks):
    population = []
    for _ in range(num_tasks):
        task_length = random.randint(*TASK_LENGTH)
        node = random.choice(nodes)
        population.append((task_length, node))
    return population

def fitness_function(solution, makespan, lambda_value=0.5):
    total_energy = sum(node.calculate_energy_consumption(makespan) for _, node in solution)
    fitness = (1 - lambda_value) * makespan + lambda_value * total_energy
    return fitness

def calculate_makespan(solution):
    node_times = {node: 0 for _, node in solution}
    for task_length, node in solution:
        node_times[node] += task_length / node.cpu_power
    return max(node_times.values())

def rfo_update(solution, best_solution, nodes):
    new_solution = []
    a = random.uniform(0, 1)
    for task_length, node in solution:
        if random.random() > 0.5:
            new_node = best_solution[random.randint(0, len(best_solution) - 1)][1]
        else:
            new_node = random.choice(nodes)
        new_solution.append((task_length, new_node))
    return new_solution

def aoa_update(solution, best_solution, nodes, t, t_max):
    new_solution = []
    for task_length, node in solution:
        r2 = random.random()
        MOP = MIN_OA + t * ((MAX_OA - MIN_OA) / t_max)
        DUL = random.random()
        if r2 < 0.5:
            new_node = best_solution[random.randint(0, len(best_solution) - 1)][1]
        else:
            new_node = random.choice(nodes)
        new_solution.append((task_length, new_node))
    return new_solution

def rfoaoa(nodes, num_tasks, iterations=100):
    population = initialize_population(nodes, num_tasks)
    best_solution = population
    best_fitness = fitness_function(population, calculate_makespan(population))
    
    for t in range(iterations):
        # RFO phase
        new_population = rfo_update(population, best_solution, nodes)
        makespan = calculate_makespan(new_population)
        new_fitness = fitness_function(new_population, makespan)
        
        if new_fitness < best_fitness:
            best_solution = new_population
            best_fitness = new_fitness
        
        # AOA phase
        new_population = aoa_update(new_population, best_solution, nodes, t, iterations)
        makespan = calculate_makespan(new_population)
        new_fitness = fitness_function(new_population, makespan)
        
        if new_fitness < best_fitness:
            best_solution = new_population
            best_fitness = new_fitness
    
    return best_solution, best_fitness

def run_simulation():
    cloud_nodes = [Node(random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH) for _ in range(CLOUD_NODES)]
    fog_nodes = [Node(random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH) for _ in range(FOG_NODES)]
    all_nodes = cloud_nodes + fog_nodes

    results = []

    for num_tasks in TASK_SETS:
        best_solution, best_fitness = rfoaoa(all_nodes, num_tasks, iterations=500)  # Increased iterations for better convergence
        makespan = calculate_makespan(best_solution)
        total_energy = sum(node.calculate_energy_consumption(makespan) for _, node in best_solution)
        
        results.append({
            "num_tasks": num_tasks,
            "makespan": makespan,
            "total_energy": total_energy
        })

    return results

# Run the simulation and store the results
simulation_results = run_simulation()
simulation_results


[{'num_tasks': 300,
  'makespan': 234.71315570358036,
  'total_energy': 3990.4412162076655},
 {'num_tasks': 600,
  'makespan': 501.1727726894255,
  'total_energy': 15569.675225356841},
 {'num_tasks': 900,
  'makespan': 778.5558780841799,
  'total_energy': 35789.76995425048},
 {'num_tasks': 1200,
  'makespan': 1084.3762927605405,
  'total_energy': 63899.94771097418},
 {'num_tasks': 1500,
  'makespan': 1298.7208774583962,
  'total_energy': 99545.20467235408}]